In [1]:
from PIL import Image
from io import BytesIO
from transformers import CLIPProcessor, CLIPModel, SegformerImageProcessor, AutoModelForSemanticSegmentation , AutoFeatureExtractor
from urllib.parse import urlparse, urlunparse
from weaviate.util import generate_uuid5
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import os
import weaviate 
import numpy as np
from numpy.linalg import norm
import json
import cv2
import base64
from time import sleep
import ast
import re

c:\Users\Jayesh\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Jayesh\AppData\Local\Programs\Python\Python39\lib\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [2]:
checkpoint = "patrickjohncyh/fashion-clip"
model = CLIPModel.from_pretrained(checkpoint)
processor = CLIPProcessor.from_pretrained(checkpoint)
seg_processor = SegformerImageProcessor.from_pretrained("mattmdjaga/segformer_b2_clothes")
seg_model = AutoModelForSemanticSegmentation.from_pretrained("mattmdjaga/segformer_b2_clothes")

def getTextEmbeddings(text):
	inputs = processor(text=text , images=Image.new('RGB' , (72 , 72)), return_tensors="pt", padding=True)
	outputs = model(**inputs , return_dict=True)
	return outputs["text_embeds"]

def getImageEmbeddingsFromPath(image_path):
	image = Image.open(image_path)
	inputs = processor(text=["dummy"] , images=image, return_tensors="pt", padding=True)
	outputs = model(**inputs , return_dict=True)	
	return outputs["image_embeds"]

def getImageEmbeddings(image):
	inputs = processor(text=["dummy"] , images=image, return_tensors="pt", padding=True)
	outputs = model(**inputs , return_dict=True)
	return outputs["image_embeds"]

def applyMask(image, mask):
	image = np.array(image)
	mask = np.array(mask)
	mask = np.stack((mask,)*3, axis=-1)
	resultant = image*mask
	resultant[mask == 0] = 255
	return resultant

def cropImage(image):
	temp = image[:, :, ::-1].copy() 
	temp = temp.astype('uint8')
	gray = cv2.cvtColor(temp, cv2.COLOR_BGR2GRAY)
	thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
	contours = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
	contours = sorted(contours, key=lambda x: cv2.contourArea(x), reverse=True)
	x,y,w,h = cv2.boundingRect(contours[0])
	crop = image[y:y+h, x:x+w]
	return crop

def segment(image, to_mask):
	inputs = seg_processor(images=image, return_tensors="pt")
	outputs = seg_model(**inputs)
	logits = outputs.logits.cpu()
	upsampled_logits = nn.functional.interpolate(
		logits,
		size=image.size[::-1],
		mode="bilinear",
		align_corners=False,
	)
	pred_seg = upsampled_logits.argmax(dim=1)[0]
	result = []
	for i in to_mask:
		mask = pred_seg.numpy().copy()
		mask[mask != i] = 0
		mask[mask == i] = 1
		item = applyMask(image, mask)
		result.append(item)
	return result

def segmentAndEmbed(image_path, to_mask):
	result = {}
	image = Image.open(image_path)
	fullImageEmbedding = getImageEmbeddings(image)
	buffered = BytesIO()
	image.save(buffered, format="PNG")
	fullImageBase64 = base64.b64encode(buffered.getvalue()).decode()
	result["fullImageBase64"] = fullImageBase64
	result["fullImageEmbedding"] = fullImageEmbedding
	segments = segment(image, to_mask)
	for i in range(len(to_mask)):
		segmentEmbedding = getImageEmbeddings(segments[i])
		segments[i] = Image.fromarray(np.uint8(segments[i]))
		buffered = BytesIO()
		segments[i].save(buffered, format="PNG")
		segmentBase64 = base64.b64encode(buffered.getvalue()).decode()
		result[f"segmentBase64_{i}"] = segmentBase64
		result[f"segmentEmbedding_{i}"] = segmentEmbedding
	return result

c:\Users\Jayesh\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\models\segformer\image_processing_segformer.py:99: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(


In [ ]:
def max_pooling(vector_list):
    vectors_array = np.array(vector_list)
    max_pooled_vector = np.max(vectors_array, axis=0) 
    return max_pooled_vector

def mean_pooling(vector_list):
    vectors_array = np.array(vector_list)
    mean_pooled_vector = np.mean(vectors_array, axis=0)
    return mean_pooled_vector

def weighted_mean_pooling(vector_list, weight_list):
    vectors_array = np.array(vector_list)
    weights_array = np.array(weight_list)
    weighted_sum = np.sum(vectors_array * weights_array[:, np.newaxis], axis=0)
    sum_of_weights = np.sum(weights_array)
    weighted_mean = weighted_sum / sum_of_weights
    return weighted_mean

def cosine_similarity(vector1, vector2):
    vector1 = np.array(vector1)
    vector1 = vector1.reshape(1, -1)
    vector2 = np.array(vector2)
    vector2 = vector2.reshape(-1, 1)
    dot_product = np.dot(vector1, vector2)
    norm1 = np.linalg.norm(vector1)
    norm2 = np.linalg.norm(vector2)
    return dot_product / (norm1 * norm2)

In [ ]:
client = weaviate.Client(url="http://localhost:8080")

In [ ]:
embeddings = getTextEmbeddings("all white outfit with white top and white pants")

In [ ]:
where_filter1 = {
    "path" : ["category"],
    "operator" : "Like",
    "valueText" : "girls"
}

In [ ]:
response1 = (
    client.query
    # .get("FlipkartProducts",["uRL", "brand", "category", "product", "price", "rating", "numberRatings", "colour", "row"])
    .get("PinterestImages", ["image", "top {... on PinterestTop { image, _additional {vector} }}"])
    .with_near_vector({"vector" : embeddings.tolist()[0]})
    .with_where(where_filter1)
    # .with_near_vector({"vector" : top_embedding})
    .with_additional(["vector", "id", "distance"])
    .with_limit(5)
    .do()
)

In [ ]:
len(response1["data"]["Get"]["PinterestImages"])

In [ ]:
for i in range(15):
    try:
        # image = response["data"]["Get"]["FlipkartProducts"][i]['image']
        # image = response1["data"]["Get"]["PinterestImages"][i]["top"][0]['image']
        image = response1["data"]["Get"]["PinterestImages"][i]['image']
        image = Image.open(BytesIO(base64.b64decode(image.split(",",1)[0])))
        image.show()
    except:
        pass

In [ ]:
list_of_embeddings = []
for i in range(len(response1["data"]["Get"]["PinterestImages"])):
    top_embedding = response1["data"]["Get"]["PinterestImages"][i]["top"][0]['_additional']["vector"]
    list_of_embeddings.append(top_embedding)

list_of_embeddings.append(embeddings.tolist()[0])

In [ ]:
weights = [0.07, 0.07, 0.07, 0.07, 0.07, 0.65]
# weights = [0.2, 0.15, 0.15, 0.5]

In [ ]:
top_mean = mean_pooling(list_of_embeddings).tolist()

In [ ]:
top_weighted_mean = weighted_mean_pooling(list_of_embeddings, weights)

In [ ]:
where_filter = {
    "path": ["category"],
    "operator": "Like",
    "valueText" : "Girls"
}

In [ ]:
response = (
    client.query
    .get("FlipkartNoSegProducts",["uRL", "brand", "category", "product", "price", "rating", "numberRatings", "colour", "brand", "image", "fit", "type"])
    .with_near_vector({"vector" : top_weighted_mean})
    .with_additional(["vector", "id", "distance"])
    .with_limit(12)
    .do()
)


In [ ]:
def text_based_rank(original_vector, products):
    product_text_embeddings = {}
    for i, product in enumerate(products):
        product_text_embedding = getTextEmbeddings(f"Name: {product['product']}, Color: {product['colour']}, Fit: {product['fit']}, Type: {product['type']}")
        product_text_embeddings[i] = product_text_embedding.tolist()[0]
    cosine_similarities = {}
    for i, embedding in product_text_embeddings.items():
        cosine_similarities[i] = cosine_similarity(original_vector, embedding).tolist()[0][0]
    cosine_similarity_ranking = sorted(cosine_similarities.items(), key=lambda x: x[1], reverse=True)
    rank_tuples = []
    for i, (rank, _) in enumerate(cosine_similarity_ranking):
        rank_tuples.append((i, rank))
    return rank_tuples


In [ ]:
text_rank = text_based_rank(embeddings.tolist()[0], response["data"]["Get"]["FlipkartNoSegProducts"])

In [ ]:
for i in range(15):
    try:
        image = response["data"]["Get"]["FlipkartNoSegProducts"][i]['image']
        # image = response1["data"]["Get"]["PinterestImages"][i]["top"][0]['image']
        # image = response1["data"]["Get"]["PinterestImages"][i]['image']
        image = Image.open(BytesIO(base64.b64decode(image.split(",",1)[0])))
        image.show()
    except:
        pass

In [ ]:
for key, value in new_rank.items():
    try:
        image = response["data"]["Get"]["FlipkartNoSegProducts"][value]['image']
        image = Image.open(BytesIO(base64.b64decode(image.split(",",1)[0])))
        image.show()
    except:
        pass

In [ ]:
user_profile_top = client.data_object.get_by_id(
    uuid=generate_uuid5("Female Profile 1 Top"),
    class_name="Customer_Profile",
    with_vector=True
)

In [ ]:
user_top_vector = user_profile_top['vector']

In [ ]:
def rank_by_popularity(products):
    popularity_scores = {}
    for i, product in enumerate(products):
        popularity_scores[i] = (float(product["rating"]))**2 * product["numberRatings"]
    popularity_rank = sorted(popularity_scores.items(), key=lambda x: x[1], reverse=True)
    rank_tuples = []
    for i, (rank, _) in enumerate(popularity_rank):
        rank_tuples.append((i, rank))
    return rank_tuples
    

In [ ]:
def rank_by_userprofile(user_top_vector, products):
    product_scores = {}
    for i, product in enumerate(products):
        product_scores[i] = cosine_similarity(user_top_vector, product["_additional"]["vector"]).tolist()[0][0]
    product_score_rank = sorted(product_scores.items(), key=lambda x: x[1], reverse=True)
    rank_tuples = []
    for i, (rank, _) in enumerate(product_score_rank):
        rank_tuples.append((i, rank))
    return rank_tuples

In [ ]:
popularity_rank = rank_by_popularity(response["data"]["Get"]["FlipkartNoSegProducts"])

In [ ]:
user_rank = rank_by_userprofile(user_top_vector, response["data"]["Get"]["FlipkartNoSegProducts"])

In [ ]:
def combine_tuples(text_rank, popularity_rank, user_rank):
    combined_rank = {}
    for i in range(len(text_rank)):
        combined_rank[i] = [i, text_rank[i][1], popularity_rank[i][1], user_rank[i][1]]
    return combined_rank


In [ ]:
combine_tuples = combine_tuples(text_rank, popularity_rank, user_rank)

In [ ]:
combine_tuples

In [ ]:
def reciprocal_rank_fusion(combine_tuples, weights):
    fused = {}
    for key, value in combine_tuples.items():
        fused_score = sum(weights[i] * (1 / (val + 1)) for i, val in enumerate(value))
        fused[key] = fused_score
    fused_ranking = sorted(fused.items(), key=lambda x: x[1], reverse=True)
    rrf_rank = {}
    for i, (rank, _) in enumerate(fused_ranking):
        rrf_rank[i] = rank
    return rrf_rank

In [ ]:
rrf_weights = [0.4, 0.25, 0.1, 0.25]

In [ ]:
rrf_rank = reciprocal_rank_fusion(combine_tuples, rrf_weights)

In [ ]:
for key, value in rrf_rank.items():
    try:
        image = response["data"]["Get"]["FlipkartNoSegProducts"][value]['image']
        image = Image.open(BytesIO(base64.b64decode(image.split(",",1)[0])))
        image.show()
    except:
        pass